In [1]:
]activate ../

Activating environment at `~/projects/borton/calcium-roi-analysis/Project.toml`


In [2]:
using Images
using FileIO
using ImageFiltering
using ImageBinarization
using ImageSegmentation
using StatsBase
using Plots

In [3]:
using PyCall
# Needed for napari
pygui_start(:qt5)
# needed to kickstart Qt5 for some reason
qapp_obj = pyimport("qtpy.QtWidgets").QApplication([""])
napari = pyimport("napari")
skimage = pyimport("skimage")

PyObject <module 'skimage' from '/Users/mrestrep/projects/borton/calcium-roi-analysis/.venv/lib/python3.8/site-packages/skimage/__init__.py'>

In [4]:
img_path = "/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs"

"/Users/mrestrep/projects/borton/calacium_images/sophie_data/tiffs"

In [5]:
function load_tiffstack(img_path)
    filenames = readdir(img_path)
    c1_files = filenames[[file[end-7:end] .== "c001.tif" for file in filenames]]
    temp = load(img_path * "/" * c1_files[1])
    voi = zeros(eltype(temp), size(temp)..., size(c1_files)[1])
    for (k, file) in enumerate(c1_files)
        filename = img_path .* "/" .* file
        img = load(filename)
        voi[:,:,k]=img
    end
    return voi
end

load_tiffstack (generic function with 1 method)

In [24]:
voi = load_tiffstack(img_path);
voi_napari = convert(Array{Float64}, voi)

In [24]:
radius = []
blob_sigmas = radius./sqrt(2)
blob_shape = [1.0, 1.0, 1.0]

In [65]:
for (i,sigma) in enumerate(blob_sigmas):
    voi_filtere[i] = imfilter(voi, Kernel.gaussian((sigma,sigma,1)));


In [124]:
println("Finding blobs")
@time blobs = blob_LoG(voi, blob_sigmas, true, blob_shape)
n_blobs = length(blobs)
println("Found  $n_blobs blobs")

Finding blobs
 31.539864 seconds (890 allocations: 21.168 GiB, 11.94% gc time)
Found  380 blobs


In [133]:
blob_x = zeros(Int64, length(blobs))
blob_y = zeros(Int64, length(blobs))
blob_z = zeros(Int64, length(blobs))
amplitude = zeros(Float64, length(blobs))
σ = zeros(Float64, length(blobs))

for i=1:length(blobs)
    coord = blobs[i].location
    blob_x[i]  = coord[1]
    blob_y[i] = coord[2]
    blob_z[i] = coord[3]
    amplitude[i] = blobs[i].amplitude
    σ[i] = blobs[i].σ
end


In [130]:
# intensity_scale = 20
h = fit(Histogram, amplitude; nbins=500)
bar(midpoints(h.edges...), h.weights)
t_val = maximum(h.weights) - 100
# savefig(output_path * "/roi_$(roi_idx-1)/julia_log_blobs_hist.png")
t = midpoints(h.edges...)[findall(h.weights.>t_val)] .+ 0.0001
thresh = t[1]
valid = amplitude .> thresh;

In [134]:
viewer = napari.view_image(voi_napari, rgb=false, name="raw")
viewer.add_image(convert(Array{Float64},voi_filtere1), name = "gauss1")
viewer.add_image(convert(Array{Float64},voi_filtere5), name = "gauss5")
viewer.add_image(convert(Array{Float64},voi_filtere7), name = "gauss7")
viewer.add_points([ blob_x[valid] blob_y[valid] blob_z[valid]], size=3, face_color = "#2aff05", edge_color = "#2aff05", name = "centroids")
viewer.add_points([ blob_x blob_y blob_z], size=3, face_color = "#2aff05", edge_color = "#2aff05", name = "centroids-all")



BoundsError: BoundsError: attempt to access 380-element Array{Int64,1} at index [Base.LogicalIndex(Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [115]:
[ blob_x[valid] blob_y[valid] blob_z[[valid]]

LoadError: syntax: incomplete: premature end of input

In [106]:
Pkg.add("Plots")

 Resolving package versions...
 Installed Adapt ─────────── v2.0.2
 Installed StructArrays ──── v0.4.4
 Installed GeometryBasics ── v0.2.15
 Installed RecipesPipeline ─ v0.1.11
 Installed PlotUtils ─────── v1.0.5
 Installed Plots ─────────── v1.5.5
 Installed Contour ───────── v0.5.4
 Installed GR ────────────── v0.51.0
  Updating `~/projects/borton/calcium-roi-analysis/Project.toml`
  [91a5bcdd] + Plots v1.5.5
  Updating `~/projects/borton/calcium-roi-analysis/Manifest.toml`
  [79e6a3ab] + Adapt v2.0.2
  [6e34b625] + Bzip2_jll v1.0.6+2
  [35d6a980] + ColorSchemes v3.9.0
  [d38c429a] + Contour v0.5.4
  [e2d170a0] + DataValueInterfaces v1.0.0
  [c87230d0] + FFMPEG v0.3.0
  [b22a6f82] + FFMPEG_jll v4.1.0+3
  [d7e528f0] + FreeType2_jll v2.10.1+2
  [559328eb] + FriBidi_jll v1.0.5+3
  [28b8d3ca] + GR v0.51.0
  [5c1252a2] + GeometryBasics v0.2.15
  [4d00f742] + GeometryTypes v0.8.3
  [cd3eb016] + HTTP v0.8.16
  [83e8ac13] + IniFile v0.5.0
  [82899510] + IteratorInterfaceExtensions v1.0.0
  [